In [79]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize, Bounds

In [80]:
df = pd.read_csv('../data/occupation_shares.csv')
df = df.iloc[:-1]
df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
df = df.set_index('MET2013')

In [81]:
# # Assuming df is your DataFrame
# def objective(rho_k, df):
#     occupations = df.columns.drop('city_total')
#     moments_real = df.mean()  # Calculate the moments of the real data
#     error = 0
#     for _, row in df.iterrows():
#         estimated_total = np.sum([row[occupation]**(1 - rho_k[i]) for i, occupation in enumerate(occupations)])
#         moments_simulated = np.mean(estimated_total)  # Calculate the moments of the simulated data
#         error += np.sum((moments_real - moments_simulated)**2)  # Minimize the squared difference between the moments
#     return error

# # Initial guess for rho_k
# rho_k_initial_guess = [0.5] * len(df.columns.drop('city_total'))

# # Define bounds
# bounds = Bounds([0]*len(rho_k_initial_guess), [1]*len(rho_k_initial_guess))

# # Use scipy's minimize function to find the best rho_k
# result = minimize(objective, rho_k_initial_guess, args=(df,), bounds=bounds)

# # The optimal rho_k is stored in the x attribute of the result
# rho_k_optimal = result.x

In [82]:
# rho_k_optimal = np.append(rho_k_optimal, 0)
# df.loc['rho_k'] = rho_k_optimal
# df.to_csv('rho_estimates.csv')

In [83]:
def get_shifter(pi_c, pi_c_prime, rho_k=0.5):
    return (pi_c / pi_c_prime)**(1 - rho_k)

# Assuming df is your DataFrame
df_new = df.copy()

for column in df.columns:
    df_new[column+'_prime'] = np.full(df.shape[0], df[column].iloc[0])
    df_new[column+'_shifter'] = df[column].apply(lambda x: get_shifter(x, df_new[column+'_prime'].iloc[0]))

df_new.drop(columns=[col for col in df_new.columns if 'prime' in col], inplace=True)
df_new.to_csv('shifter_estimates.csv')


/tmp/ipykernel_5613/670960256.py:2: RuntimeWarning: invalid value encountered in scalar divide
  return (pi_c / pi_c_prime)**(1 - rho_k)
/tmp/ipykernel_5613/670960256.py:2: RuntimeWarning: divide by zero encountered in scalar divide
  return (pi_c / pi_c_prime)**(1 - rho_k)
